In [ ]:
import pandas as pd
from dateutil.relativedelta import *
from pmdarima.arima import auto_arima
from pmdarima.arima import ADFTest
from sklearn.metrics import r2_score, mean_squared_error
import math
import matplotlib.pyplot as plt
import pypyodbc as podbc
from sqlalchemy import create_engine
import urllib

In [ ]:
connet2db = podbc.connect("Driver={SQL Server Native Client 11.0};"
                     "Server=DESKTOP-ES9JH7P;"
                     "Database=SharpParts10;"
                     "Trusted_Connection=yes;")

In [ ]:
data = pd.read_sql_query('''SELECT * FROM [dbo].[monthly_sales_data]''', connet2db)

In [ ]:
data

In [ ]:
data.dtypes

In [ ]:
# data["month"] = data["month"].apply(lambda x: datetime.strptime(x, "%Y-%m"))
data.set_index("month", inplace=True)

In [ ]:
data

In [ ]:
data.shape

In [ ]:
data.dtypes

In [ ]:
data

In [ ]:
data["honda"]

In [ ]:
plt.figure(figsize=(30, 8))
for column in data.columns:
    plt.plot(data[column], marker="o", label=column.upper())
plt.title("Sales of the Product")
plt.xlabel("Months")
plt.ylabel("Sales (LKR)")
plt.grid()
plt.legend()
plt.show()

In [ ]:
adf_test = ADFTest()
for column in data.columns:
    print(adf_test.should_diff(data[column]))

In [ ]:
def split_train_test(training_sample_count, data, plot=True):
    train = data[:training_sample_count]
    test = data[training_sample_count:]
    if plot:
        plt.plot(train)
        plt.plot(test)
    
    return train, test


In [ ]:
def train_arima(training_data, summary=True):
    arima_model = auto_arima(
        training_data,
        start_p=0, d=1, start_q=0,
        max_p=5, max_d=5, max_q=5,
        start_P=0, D=1, start_Q=0, 
        max_P=0, max_D=5, max_Q=5,
        m=12, seasonal=True,
    )
    if summary:
        print(arima_model.summary())
    
    return arima_model

In [ ]:
def predict(arima_model, prediction_period, index):
    return pd.DataFrame(arima_model.predict(n_periods=prediction_period), index=index, columns=["prediction"])

In [ ]:
quoted = urllib.parse.quote_plus("Driver={SQL Server Native Client 11.0};"
                     "Server=DESKTOP-ES9JH7P;"
                     "Database=SharpParts10;"
                     "Trusted_Connection=yes;")

In [ ]:
for column in data.columns:
    training_sample_count = round(data.shape[0] * 0.75)
    train, test = split_train_test(training_sample_count, data[column], plot=False)
    model = train_arima(train, summary=False)
    # prediction_period = data.shape[0] - training_sample_count
    prediction_period = 30
    index = pd.date_range(train.index[-1] + relativedelta(months=1), periods=prediction_period, freq='MS')
    prediction = predict(model, prediction_period, index)

    engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

    prediction.to_sql(f'{column}_sales_prediction', schema='dbo', con = engine, if_exists = 'append', index_label="timestamp")

    plt.figure(figsize=(30, 8))
    plt.plot(train, marker="o", label="Train")
    plt.plot(test, marker="o", label="Test")
    plt.plot(prediction, marker="o", label="Prediction")
    plt.title(f"Real Sales and Forecasted Sales of {column.upper()}")
    plt.xlabel("Month")
    plt.ylabel("Sales (LKR)")
    plt.legend(loc='upper right')
    plt.grid()
    plt.show()

In [ ]:
prediction